In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from afinn import Afinn
from dash import Dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output

In [ ]:
# https://nealcaren.org/lessons/wordlists/
afinn = Afinn(language='en')

In [ ]:
social_media_df = pd.read_csv('hw05-data/YInt Social Media Data/YInt.csv')
social_media_df['time'] = pd.to_datetime(social_media_df['time'])

In [ ]:
# social_media_df['sentiment_score'] = social_media_df['message'].apply(afinn.score)
def apply_sentiment_analysis(message):
    return afinn.score(str(message))
social_media_df['sentiment_score'] = social_media_df['message'].apply(apply_sentiment_analysis)

In [ ]:
# group messages by 1 hour
social_media_df['time'] = pd.to_datetime(social_media_df['time'], errors='coerce').dt.floor('1h')

In [ ]:
messages_by_neighborhood = social_media_df[['time', 'location', 'sentiment_score']].groupby(['time', 'location']).median().reset_index()

In [ ]:
def int_to_neighborhood(i):
    match i:
        case(1):
            return 'Palace Hills'
        case(2):
            return 'Northwest'
        case(3):
            return 'Old Town'
        case(4):
            return 'Safe Town'
        case(5):
            return 'Southwest'
        case(6):
            return 'Downtown'
        case(7):
            return 'Wilson Forest'
        case(8):
            return 'Scenic Vista'
        case(9):
            return 'Broadview'
        case(10):
            return 'Chapparal'
        case(11):
            return 'Terrapin Springs'
        case(12):
            return 'Pepper Mill'
        case(13):
            return 'Cheddarford'
        case(14):
            return 'Easton'
        case(15):
            return 'Weston'
        case(16):
            return 'Southton'
        case(17):
            return 'Oak Willow'
        case(18):
            return 'East Parton'
        case(19):
            return 'West Parton'

In [ ]:
damage_report_df = pd.read_csv('hw05-data/Damage Reports/mc1-reports-data.csv')
damage_report_df['time'] = pd.to_datetime(damage_report_df['time'])
# damage_report_df['location'] = damage_report_df['location'].astype(str)

    
damage_report_df['time'] = pd.to_datetime(damage_report_df['time'], errors='coerce').dt.floor('1h')
damage_report_df['location'] = damage_report_df['location'].apply(int_to_neighborhood)
grouped_damage = damage_report_df.groupby(['location', 'time']).max().reset_index()
grouped_damage['overall_damage'] = grouped_damage[['sewer_and_water', 'power', 'roads_and_bridges', 'medical', 'buildings']].mean(axis=1)
grouped_damage

In [ ]:
merged_sentiment_shake = messages_by_neighborhood.merge(grouped_damage[['time', 'location', 'shake_intensity', 'overall_damage']], how='inner', on=['time', 'location'])
merged_sentiment_shake

In [ ]:
traces = {}
for location in merged_sentiment_shake['location'].unique():
    filtered_df = merged_sentiment_shake[merged_sentiment_shake['location'] == location]
    traces[location] = []
    traces[location].extend(
        [go.Scatter(
            x = filtered_df['time'],
            y = filtered_df['sentiment_score'],
            name = f"mean sentiment score"
        ),
        go.Scatter(
            x = filtered_df['time'],
            y = filtered_df['overall_damage'],
            name = f"mean damage score"
        ),
        go.Scatter(
            x = filtered_df['time'],
            y = filtered_df['shake_intensity'],
            name = f"max shake intensity"
        )]
    )

In [ ]:
fig = go.Figure()
location = "Southton"
for trace in traces[location]:
    fig.add_trace(trace)
fig.update_layout({
    "title": f"Damage vs Shake Intensity vs Social Media Sentiment Score for {location}"
}
)
fig.show()

In [ ]:
figure = go.Figure()
app = Dash(__name__)
app.layout = html.Div([
    dcc.Dropdown(
        merged_sentiment_shake['location'].unique(), 
        'Southton', 
        id='selected_location',
        style={'width':'50%', 'margin-left':'auto'}
    ),
    dcc.Graph(id='my-graph', figure=figure),
])
@app.callback(
    Output(component_id='my-graph', component_property='figure'),
    Input(component_id='selected_location', component_property='value')
)
def update_graph(selected_location):
    print(f'updating to {selected_location}')
    figure = go.Figure()
    for trace in traces[selected_location]:
        figure.add_trace(trace)
    figure.update_layout({
        "title": f"Damage vs Shake Intensity vs Social Media Sentiment Score for {selected_location}"
    })
    return figure

app.run_server(debug=True)